#### Imports

In [34]:
import json
import nltk
from nltk.collocations import *
import datetime as dt
import locale
import spacy
from tqdm import tqdm
import pprint
import pandas as pd
from ast import literal_eval
import os
import gensim
import gensim.corpora as corpora
from pprint import pprint


In [10]:
#!python -m spacy download de_core_news_lg

#### Read Data

In [22]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#change cwd
%cd drive/MyDrive/Work/Frontline/data

In [20]:
# paths
FILTERED_PATH="filtered_4_26"

In [26]:
## FOR NOW: USE SMALL DATA SET
df=pd.read_csv("sample.csv")

### Topic Modelling


#### Prepare Data


In [27]:
# custom module
import preprocessing

In [28]:
# Load model
spacy_mod = spacy.load("de_core_news_lg", disable=['ner', 'parser', 'tagger'])

In [29]:
# read custom stopwords

# open list of custom stopwords
custom_stop_words= open("custom_stopwords.txt").read().split()

# add custom stopwords to model
for word in custom_stop_words:
  spacy_mod.Defaults.stop_words.add(word)

In [30]:
# convert corpus to language object
spacy_lang = []
for i, doc in tqdm(df.iterrows()): 
  spacy_lang.append(spacy_mod("".join(doc['text'])))

50it [00:01, 26.19it/s]


In [31]:
# preprocess: remove stopwords
spacy_cleaned = []
for doc in tqdm(spacy_lang): 
    spacy_cleaned.append(preprocessing.preprocess(doc, remove_ent=True))

100%|██████████| 50/50 [00:00<00:00, 869.17it/s]


#### Topic Analysis

In [33]:
# Create Dictionary
id2word = corpora.Dictionary(spacy_cleaned)

# Create Corpus
texts = spacy_cleaned

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [37]:
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id2word, num_topics=num_topics)
# Print the key words for each topic
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.016*"gewalt" + 0.011*"frauen" + 0.011*"opfer" + 0.005*"frau" + '
  '0.005*"häuslicher" + 0.004*"prozent" + 0.004*"mann" + 0.004*"männer" + '
  '0.003*"straftaten" + 0.003*"menschen"'),
 (1,
  '0.009*"frau" + 0.009*"gewalt" + 0.006*"mann" + 0.004*"opfer" + '
  '0.004*"frauen" + 0.004*"häuslicher" + 0.004*"polizei" + 0.002*"kinder" + '
  '0.002*"leben" + 0.002*"staatsanwaltschaft"'),
 (2,
  '0.008*"frauen" + 0.007*"gewalt" + 0.007*"opfer" + 0.007*"frau" + '
  '0.006*"mann" + 0.005*"polizei" + 0.005*"männer" + 0.004*"häuslicher" + '
  '0.003*"leben" + 0.003*"täter"'),
 (3,
  '0.022*"frauen" + 0.017*"gewalt" + 0.007*"opfer" + 0.006*"frau" + '
  '0.005*"männer" + 0.005*"häuslicher" + 0.004*"menschen" + 0.004*"prozent" + '
  '0.003*"polizei" + 0.003*"mann"'),
 (4,
  '0.012*"frau" + 0.012*"gewalt" + 0.009*"frauen" + 0.006*"mann" + '
  '0.005*"häuslicher" + 0.004*"polizei" + 0.004*"fälle" + 0.003*"kinder" + '
  '0.003*"opfer" + 0.003*"mutter"')]


In [36]:
## VISUALiZATION NOT WORKING ##

# import pyLDAvis
# import pyLDAvis.gensim
# import pickle 
# pyLDAvis.enable_notebook()
# pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

END OF CODE